In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from scipy.spatial.distance import cosine
import pandas as pd
import torch
import torch.nn as nn
import torch.hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Directory containing our images
images_directory = '/content/drive/MyDrive/test_dataset'
# "..\corpus_lipade\presse\photos\jpg"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pre Processing the Data to fit ResNet



In [ ]:
def preprocess_image(image_path):
    input_image = Image.open(image_path)
    preprocess = transforms.Compose([
        # Convert the image to grayscale but keeping all 3 channels
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(256),
        transforms.CenterCrop(224),

        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model
    return input_batch

def get_features(image_batch, model):
    if torch.cuda.is_available():
        image_batch = image_batch.to('cuda')
    with torch.no_grad():
        features = model(image_batch)
    return features.cpu().numpy()


def compare_images(feature1, feature2, threshold):
    similarity = 1 - cosine(feature1.flatten(), feature2.flatten())
    return similarity > threshold

# Load the pre-trained ResNet model

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

# Removing the last classification layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()
if torch.cuda.is_available():
    model.to('cuda')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Extracting features of all images

In [ ]:
image_features = {}
for image_name in os.listdir(images_directory):
    if image_name.endswith(".jpg"):
        image_path = os.path.join(images_directory, image_name)
        image_batch = preprocess_image(image_path)
        features = get_features(image_batch, model)
        image_features[image_name] = features

# Comparing each image to every other image

In [ ]:
Threshold = 0.88
similar_images = {}
for img1, features1 in image_features.items():
    similar_images[img1] = []
    for img2, features2 in image_features.items():
        if img1 != img2 and compare_images(features1, features2,Threshold):
            similar_images[img1].append(img2)

In [ ]:
def calculate_metrics(ground_truth, predictions):

    # Initializing
    tp, fp, fn = 0, 0, 0

    # Converting ground truth to a dict
    ground_truth_dict = ground_truth.set_index('Reference Image')['Similar Images'].to_dict()
    ground_truth_dict = {k: set(str(v).split(', ')) for k, v in ground_truth_dict.items()}

    # for each reference image in the predictions
    for index, row in predictions.iterrows():
        ref_image = row['Reference Image']
        predicted_similar = set(str(row['Similar Images']).split(', '))

        # Get the corresponding ground truth
        actual_similar = ground_truth_dict.get(ref_image, set())

        # Calculate TP, FP, and FN
        tp += len(predicted_similar.intersection(actual_similar))
        fp += len(predicted_similar - actual_similar)
        fn += len(actual_similar - predicted_similar)

    # Calculate precision, recall, and F1 score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [ ]:
# Load the datasets
file_path_ground_truth = '/content/drive/MyDrive/ground_truth.xlsx'
file_path_predictions = '/content/drive/MyDrive/similar_images.xlsx'
ground_truth_df = pd.read_excel(file_path_ground_truth)
predictions_df = pd.read_excel(file_path_predictions)

# Calculate the metrics
precision, recall, f1 = calculate_metrics(ground_truth_df, predictions_df)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.048879837067209775
Recall: 0.35294117647058826
F1 Score: 0.08586762075134169


# Historique des resultats

## **RESNET 101**
# Threshold: 0.9
* Precision: 0.05851063829787234
* Recall: 0.16176470588235295
* F1 Score: 0.0859375


# Threshold: 0.89
* Precision: 0.05963302752293578
* Recall: 0.19117647058823528
* F1 Score: 0.09090909090909093

# Threshold: 0.88
* Precision: **0.061567164179104475**
* Recall: 0.2426470588235294
* F1 Score: **0.0982142857142857**

# Threshold: 0.87
* Precision: 0.056338028169014086
* Recall: 0.27941176470588236
* F1 Score: 0.09376927822331894

# Threshold: 0.85
* Precision: 0.043254817987152035
* Recall: **0.3713235294117647**
* F1 Score: 0.0774836977368623


## **RESNET 152**
# Threshold: 0.88
* Precision: 0.06626506024096386
* Recall: 0.2426470588235294
* F1 Score: 0.10410094637223975

# Threshold: 0.85
* Precision: 0.048879837067209775
* Recall: 0.35294117647058826
* F1 Score: 0.08586762075134169